In [1]:
'''
東森新聞雲爬蟲練習
練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。
'''
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
'''
以財經新聞為例，先連結到財經新聞網頁，複製其URL。
'''
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

In [4]:
'''
用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。
'''
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


In [ ]:
#到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] SEMI估2024年前台灣再蓋11座晶圓廠

▲台積電在南科持續擴廠。（圖／資料照）
記者周康玉／台北報導
報告指出，2020年12吋晶圓廠投資較去年成長13%，突破2018年前高、創歷史新高，在新冠病毒疫情加速全球數位轉型的推動下，成長態勢可一路持續到2022年，預計2023年將再創高峰，為半導體產業的豐收年。
除了雲端服務、伺服器、筆記本電腦、遊戲和醫療科技等需求動能外，連結性、大型資料中心和大數據發展的5G、物聯網（IoT）、汽車、人工智慧（AI）和機器學習的快速發展也功不可沒。
SEMI全球行銷長暨台灣區總裁曹世綸表示，新冠疫情幾乎加速所有產業數位轉型的腳步，重塑我們工作與生活的方式，而創紀錄的支出預測以及38座新晶圓廠正是半導體作為先進科技發展基石的最佳例證，相關技術除了將帶動這波轉型持續前行，也有望讓世界面臨的巨大挑戰皆能迎刃而解。
半導體晶圓廠投資2021年將繼續增長，唯增速將較前一年同比放緩4%。產業週期再次上演，2023年攀上700億美元歷史新高的前後，2022年將溫和緩降，至2024年再次小幅下滑，雖有小幅波動，但整體投資的規模則是逐年拉高。
中國占全球12吋晶圓產能比重則快速增加，從2015年的8%持續爬升、2024年將達20%，預計2024年達到每月150萬片（wpm）。中國旗下企業組織正在加速產能投資，相關企業2020年占中國晶圓廠產能43%左右，預計2022年將達到50%，2024更將爬升至60%。
相對之下，日本的12吋晶圓產能在全球占比繼續下探，從2015年的19%跌至2024年的12%；美洲也下降走勢，預估從2015年的13%降至10%。
區域最大支出國寶座由韓國拿下，投資額在150億美元至190億美元之間，台灣則以12吋晶圓廠投資額140億美元至170億美元緊追在後，其次是中國，投資額在110億美元至130億美元之間。
----------------------------------------------------------------------

[1] 「監管機制太嚴」螞蟻上市暫緩？　專家：正好修改失誤避免風險

▲螞蟻金服。（圖／視覺中國）
記者任以芳／綜合報導


記者張靖榕／綜合外電報導
2020美國總統大選當地時間周二（3日）登場，美股市場對選後情勢明朗化的期待，延續周一漲勢，早盤各家高開一路收高，三大指數漲幅均超過1.7％。
道瓊工業指數終場上漲554.98點，或2.06％，以27,480.03點作收；那斯達克指數上漲202.96點，或1.85％，以11,160.57點作收；標普500指數則勢終場上漲58.92點，或1.78％，收在3,369.16點。
▲美股本周漲市持續。（圖／路透社）
全美股市情緒高漲，眼看可能再迎來新的牛市，不過有分析師警告，市場現在有兩大盲點。華爾街一派認為，股市已在超前慶祝民主黨白宮、參議院雙贏，後續將有規模比川普政府更大的激勵措施推出。另一派則認為，市場從上周的拋售中反彈未止，目前還難有定論。
另一方面則是美國金融市場普遍認為民主黨會一舉翻盤，甚至有全面執政的可能，投資者開始湧入民主黨先前規劃提出的激勵措施中，明顯會因此受益的股票。然而，新冠病毒與選舉的風向極亂，結果難測，風險卻被忽略。
----------------------------------------------------------------------

[10] 澳洲人瘋狂存錢…8個月戶頭猛增2.5兆　央行宣布降息到0.1％

▲澳洲民眾不看好經濟景氣，瘋狂存錢。（圖／pixabay）
圖文／CTWANT
澳洲的儲蓄金額，在今年2月至9月間增加了1千億澳幣（約新台幣2兆540億元），原因在於人民覺得新冠肺炎將導致經濟不景氣，故選擇儲蓄做為避險方式。另外，澳洲聯邦準備銀行（RBA）也宣布將利息從0.25%調降到0.1%，利息收入將大幅減少之際，儲蓄率卻創下新高。
根據澳洲金融監管局（APRA）的調查，9月的儲蓄額達1兆900億澳幣，比前年同期增加1,150億澳幣，光是9月新儲蓄額，就有165億澳幣。聯邦政府的經濟補貼金等措施抵銷家庭收入的銳減，並增加儲蓄額的提高。澳大利亞國民銀行（NAB）市場經濟部門負責人表示，隨著隨著薪金補貼的逐步終止，若就業市場能持續恢復，當前的儲蓄可能會轉換為消費。
另外根據金融服務比較網RateCity的資訊，目前支付的40萬澳幣定存利息已經是2月時的一半，4大銀行6個月定存的利息是0.73%，比起新冠肺炎前的1.56%已是大幅下降。
►粉絲一用6年沒變心
------------------

記者周康玉／台北報導
全球晶圓代工產能吃緊，聯電、世界先進等成熟製程都已調漲報價，IC設計大廠瑞昱（2379）副總黃依瑋今（3）日透露，這情況會持續到明年第一季，遇特殊情況，會把客戶一起帶來跟晶圓廠「討論」，看怎樣的量與價是三方可接受的。換言之，客戶為了趕上交期，願提高出貨量，晶圓代工廠也會為了維持更穩定的高稼動率，願意讓步，瑞昱居中獲得代工廠產能，可說是三贏局面。
瑞昱今日公布庫存水位已從89天降至62天，黃依瑋表示，從第3季起，已有一部分供貨是來自庫存，此情況在第4季將延續，庫存水位越降低，將造成供貨限制緊張。
瑞昱主要投片的代工廠是台積電和聯電，但在全球晶圓代工產能吃緊情況下，也開始尋求其他晶圓廠，加大晶圓供貨能力。
面對晶圓代工廠報價漲價，瑞昱講價錢有一套。「價錢是談出來的！」黃依瑋表示，當需求面高時，供應商確實在價格談判上在較有利位置，但遇特殊情況，瑞昱會把客戶帶來跟晶圓廠討論，看怎樣的量與價錢是大家可接受的，一起討論，也因此，這波漲價潮對瑞昱的毛利影響在可控範圍內。
----------------------------------------------------------------------

[22] 國泰金確定不辦尾牙！　富邦金、新光金備妥方案

記者陳依旻／台北報導
疫情持續，筆電大廠仁寶開第一槍決定今年不辦尾牙，其他電子大廠觀望中，金控公司今年舉辦與否各界關注，目前為止，國泰金控確定不舉辦。
國泰金指出，考慮疫情相關風險，故今年度不舉辦群聚型尾牙，但會以不同形式，支持餐飲業；富邦金控表示，目前開始規劃籌辦謝年會，不過會視疫情情況調整，尚未最後定案。
「還是會舉辦尾牙！」新光金控發言人強調，會朝小型方向規劃；台新金控也是會照舊舉行，總經理林維俊昨日下午表示，是向衛福部請教後的結果；中信金控目前則不確定。
----------------------------------------------------------------------

[23] 專職研發人員教育訓練費可抵稅　會計師提醒：2月到5月底記得申請

▲經濟部、財政部全職研發人員教育訓練費可列入投資抵減營所稅。（圖／視覺中國）
記者吳靜君／台北報導
資誠聯合會計師事務所表示，經濟部、財政部公告修正「研究發展支出適用投資抵減辦法」，新增全職研發人員參與研究發展專業知識

圖、文／福倉建築機構提供
福倉建築機構深耕建築界15年，以「不蓋豪宅只蓋好宅」與「用溫度蓋房，用心愛家」的職人精神貫徹始終。二期福倉雲硯昇華進化成「訂製宅3.0」，從客戶生活的理想作思考，規劃36-41坪3房的適居格局，推出專門顧問團隊負責設計變更，從需求評估、室內規劃到監工驗屋等一條龍服務，只為讓未來住戶安心滿意。
福倉雲硯近中路三大公園之一「水秀公園」、中興國中、文山國小學區，貼心建材規劃，結構制震採用美國航太認證Taylor制震器，101指定御用台灣水泥、減噪木地板、三協氣密窗和LOW-E玻璃、日系Panasonic整體規劃廚俱，並由瑞士SGS建物生產履歷嚴格把關每一階段工程品質。
敬請把握機會訂製心中夢想好宅！洽詢電話：（03）301-0066。
----------------------------------------------------------------------

[36] 快訊／東洋代理進口新冠疫苗生變！　卡在德國BNT與我疾管署「未達成協議」

記者姚惠茹／台北報導
東洋（4105）今（3）日宣布，日前取得德國BioNTech(BNT)新冠疫苗在台銷售的「有條件授權書」後，經20多天密集地與BNT、衛生福利部疾病管制署協商討論，雙方對數量及價格仍有落差，未能達成一致協議，因此東洋將無法如期代理新冠疫苗進口。
東洋表示，今年10月12日取得德國BNT授權，可就疫苗交貨數量、時間、價格與台灣主管機關協商，並積極扮演BNT與政府間的溝通橋樑，分別就最新臨床數據、採購數量、價格、冷鏈運送方式等與政府進行討論。
▲東洋藥品董事長林全。（圖／記者屠惠剛攝）
東洋指出，溝通協商期間，雖然東洋努力居中協調、拉近差距，但是政府與德國BNT在採購數量及價格上仍有落差，截至授權協商期滿並未達成共識。
面對全球新冠疫情肆虐，東洋重申，未來仍將配合政府防疫政策，持續在公司能力範圍內，善盡企業社會責任，為維護國人健康努力。
----------------------------------------------------------------------

[37] 出社會5年沒錢！他怨「低薪、狂加班」害婚事告吹　專家揭現實面

▲讀者來信，認為自己很努力工作，收入卻不如預期。（圖／123RF）
這兩天有位年輕讀者，大約是82年次，算一算應該也27、

▲由左至右：國泰投信發言人吳惠君、國泰投信總經理張雍川、基金經理人蘇鼎宇。（圖／記者陳心怡攝）
記者陳心怡／台北報導
台灣5G今年7月正式開跑，高通預估至2035年，台灣直接或間接創造的5G相關總產值高達1,340億美元。國泰投信總經理張雍川表示，5G是未來十年的投資主流，而全球最強5G供應鏈就在台灣，因此國泰投信推出台股第一檔5G科技主題ETF—國泰台灣5G+ETF(00881)，掌握全球財富新核心。
張雍川表示，全球最強5G供應鏈在台灣的主因有三，第一，技術面領先，台廠握有全球最強的半導體供應鏈、5G手機及資通訊基礎建設等關鍵技術；第二，政策面做多，政府帶頭推動5G行動計畫，產值預估達新台幣500億；第三，投資面勝出，台廠具高競爭優勢，及中美角力下台廠左右逢源，在在催化臺灣5G+通訊指數表現超越全球5G市場。
國泰台灣5G+ ETF11/18~11/24展開募集，IPO每股發行價格15元，一年配息兩次（7、12月），12/10正式掛牌上市，該檔ETF追蹤的是臺灣5G+通訊指數，係以紐約證券交易所及納斯達克上市的專業金融數據公司FactSet的產業分類作為篩選，涵蓋45個5G 相關子產業，完整掌握5G及未來6G、甚至7G的投資機會。
此外，國泰台灣5G+ ETF指數成分股挑選經層層把關：第一層，選高純度5G股，只挑臺灣5G產業營收佔比>50%的上市櫃股票；第二層，挑流動性佳個股，近三個月日均成交額達5千萬以上；第三層，買獲利績優股，只投資近四季累積EPS>0的公司股票。經層層篩選後，最終納入前30大5G績優股且依市值加權。
國泰台灣5G+ETF經理人蘇鼎宇表示，5G網路連接數才正要開始加速，投資5G，時間早不如時機巧，預估2021年才是5G真正爆發期，尤其美中貿易衝突加速台廠得利，建議投資人佈局台灣5G關鍵供應鏈，搶佔新時代的關鍵先機。
----------------------------------------------------------------------

[47] 代銷保證預售屋可貸9成！網揭背後「暗黑話術」：房貸+信貸真的硬

記者黃可昀／綜合報導
買房必須先準備2到3成自備款，很多人就是卡在頭期款不足這關，預售屋的好處是前期還款壓力較低，房貸成數也較高，所以近幾年相當熱門。一名女子最近在看預售屋，「每個代銷都說基本上可以貸到9成」，她聽

▲大同新任董事長林文淵。（圖／記者李毓康攝）
記者陳心怡／台北報導
大同昨（2）日首次舉行新任董事會，由東森電視董事長林文淵接任大同董事長，林文淵在會後記者會提到，會儘早發放股利給股東，請股東有信心，不過今（3）日大同股價卻開高走低，早盤一度漲2%至21.15元，隨後卻翻跌來到20.15元，投資人心情大洗三溫暖。
林文淵昨天表示，大同已經19年沒有發放現金股利，目前不能保證什麼時候可以開始發放股利，不過會盡最快的速度發放，過往上市公司的經營策略，就是保留盈餘少，盡可能發放現金股利給股東。
大同股價今天早盤，一度受到激勵，最高來到21.15元，但一個小時後就翻跌下挫，最低跌逾2%，來到20.15元。
----------------------------------------------------------------------

[60] 彰銀明年設置50家雙語分行　2028年完成全雙語

▲金管會、國發會與財政部視察彰銀台中雙語分行。（圖／記者吳靜君攝）
記者吳靜君／台北報導
彰化銀行響應2030雙語國家政策發展藍圖推動金融服務雙語化，陸續在台北、台中及高雄設置三家雙語示範分行，今（3）日由董事長凌忠嫄、總經理黃瑞沐陪同金管會副主委邱淑貞、國發會副主委游建華及財政部次長莊翠雲等前往視察。
彰化銀行台中分行位在台中市南區，鄰近烏日、溪南工業區、台中精密機系科技創新園區、中興大學及中山醫學大學，外籍客戶包含企業外籍員工及大專院校的外籍師生；台中分行為強化外籍客戶的金融服務體驗，設立英語服務專線讓金融大小事溝零距離。
凌忠嫄表示，彰銀在雙語分行建置之初，就透過大數據分析，掌握外籍人士在各種通路中經常進行的交易與需求，針對臨櫃表單、自動櫃員機、網路及行動銀行的交易畫面雙語化，上線之後更透過分行與外籍客戶的互動，進一步進行相關設施改善與調整，具體落實友善雙語金融環境。
目前已經在總部分行、台中分行、高雄分行設置雙語服務，未來加快建置時程，在港口、觀光景點等區域，預計明（2021）年在設置50家雙語分行，並提前在2028年前國內185家分行雙語化。
▼彰銀台中雙語分行。（圖／記者吳靜君攝）
----------------------------------------------------------------------

[61] 王美花談美國選後新局：台

記者王致凱／綜合報導
蘋果公司宣布11月10日將舉行發表會。外界預計此次將發表全新搭載蘋果自家晶片的Mac系列電腦。
蘋果周一宣布發表會將於美西時間10日上午10時（台灣時間11日凌晨2時）開始，並且將在蘋果官方網站上直播，邀請函上的標語則是「One More Thing」。
蘋果稍早前表示，計畫發表搭載基於ARM架構設計的自家晶片Mac電腦，以取代過去的英特爾晶片。蘋果的iPhone手機系列向來使用自家處理器晶片，據稱有助於延長電池壽命並提升性能。
----------------------------------------------------------------------

[74] 快訊／大選前最後交易日…美股3大指全面反彈　道瓊跳漲355點

記者王致凱／綜合報導
美國大選投票日前最後一個交易日，美股周一（2日）從上月陰霾中反彈，道瓊工業指數開盤跳漲355點或1.3％，標普500指數上漲1.1％，那斯達克綜合指數則上漲0.9％
道瓊期指稍早前上漲435點，預示開盤將上漲約400點。標普500期指也上漲1.48％，那斯達克期指則上漲1.31％。
受美國財政刺激方案協商破裂、新冠感染率飆高雙重影響，美股道瓊上周下跌6.5％，標普500下跌5.6％，創下3月份以來最大單周跌幅。那指上周跌幅也達5％以上。
►雙11狂歡購物節！「理膚寶水B5全面修護霜」限時下殺788元！ 
----------------------------------------------------------------------

[75] 今彩539頭獎今晚開出！　一注800萬「獎落新竹」

▲今彩539。（示意圖／記者董美琪攝）
記者楊庭蒝／綜合報導
今彩539第109000263期2日晚間開獎，中獎號碼 12、29、31、36、39  。頭獎由新竹市北區文雅里的「鈞鈞彩券行」一注獨得，得主可拿800萬元獎金。
台彩官網公布，第109000263期開出800萬元頭獎1注、2萬元貳獎219注、300元參獎7,956注、50元肆獎82,476注。頭獎由新竹市北區文雅里的「鈞鈞彩券行」開出。
第109000088期威力彩中獎號碼，第一區由小到大依序為「04、12、15、32、37、38」，第二區04，頭獎與貳獎摃龜；參獎共10注中獎，每注可得15萬元；肆獎共27注中獎，每

▲第13屆優秀器官勸募機構暨人員頒獎典禮。（圖／東洋提供）
記者姚惠茹／台北報導
台灣東洋（4105）今（2）日出席由財團法人器官捐贈移植登錄中心舉辦的「第13屆優秀器官勸募機構暨人員頒獎典禮」，由衛生福利部次長薛瑞元頒發「企業公益獎」，藉此肯定台灣東洋響應社會公益、鼓舞員工了解器官捐贈的大愛精神。
台灣等待器捐人數約有9900人，而實際捐贈人數每年只有300例，兼任器捐登錄中心董事長的衛福部次長薛瑞元期勉社會持續宣導器捐理念，讓有需要的人與家庭不要承受太多煎熬，而為了推動理念，器捐登錄中心則是首度辦理「企業公益獎」，台灣東洋更是首批獲獎的公益企業之一。
台灣東洋表示，公司今年以「永續共榮愛無限」為主題，舉行企業年度家庭日活動，並安排器捐登錄中心宣導理念，透過專業解說，不少台灣東洋員工把感動化為行動，簽署「器官捐贈同意書」，讓器捐理念持續廣傳，因此獲得「企業公益獎」的肯定與鼓勵。
▲左起為台灣東洋人資長張人仰、衛生福利部次長薛瑞元。（圖／東洋提供）
台灣東洋人資長張人仰到現場代表受獎，並對捐贈者家屬楊淑姿分享女兒陳俐蓉車禍意外後決定器捐的故事感到十分動容，並強調台灣東洋的使命之一是提升人類健康福祉，除了透過同仁平時各自在其專業領域中達成，也透過鼓勵員工參與各種公益活動，以善盡社會企業責任並落實企業文化。
台灣東洋強調，推動友善器官捐贈環境是器捐登錄中心多年努力的工作，台灣東洋願持續響應理念宣導，讓愛與希望的種子深根發芽，也盼每一個生命故事都能正向影響不同的生命，為利他社會貢獻心力。
----------------------------------------------------------------------

[89] 超狂！他用「瓦斯桶」存錢　零錢灑滿地超療癒…網歪樓：洞怎麼挖

▲竟然有人用瓦斯桶存錢，網友一看全驚呆。（圖／翻攝自Facebook／爆廢1公社）
記者施怡妏／綜合報導
每個人都有自己的一套存錢方式，有些人喜歡把錢存在銀行裡，有些人則喜歡存在撲滿裡，每天一點一點的累積，時間久了也是一筆數目可觀的金錢。一名網友分享好友的存錢方式，不是存在銀行，也不是存在小豬撲滿，竟然是存在「瓦斯桶」。許多網友看到後驚呼，「超級狂，算錢算到瘋掉」。
文章貼出短短3小時，立刻吸引3000名網友按讚，留言數也高達300則，「這被闖空門還不會被偷，真要偷還搬不太

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)